In [60]:
import pandas as pd
import plotly.express as px

In [71]:
df = pd.read_parquet("../data/outputs/plots/plot_metrics.parquet")
df

,max,min,range,mean,median,sd,var,cv,crr,skew,...,norm_fhd,cv_inside,cv_inside_p,cv_ppi,cv_vad,site,site_type,variant,grid_agg,grid_size
plot,,,,,,,,,,,,,,,,,,,,,
AGG_O_01_P1,16.275999,0.000000,16.275999,4.428016,4.555000,4.093416,16.756056,0.924436,0.272058,0.245465,...,0.824705,1.051162,1.051162,0.111321,1.511392,AGG_O_01,AGG,default,none,0
AGG_O_01_P1,16.275999,0.000000,16.275999,6.716327,7.568000,3.647536,13.304515,0.543085,0.412652,-0.601755,...,0.860111,0.848656,0.848656,0.129149,0.997475,AGG_O_01,AGG,rnw,none,0
AGG_O_01_P1,16.275999,0.000000,16.275999,4.428016,4.555000,4.093416,16.756056,0.924436,0.272058,0.245465,...,0.821678,1.066437,1.066437,0.110900,1.522600,AGG_O_01,AGG,iw,none,0
AGG_O_01_P1,16.275999,0.000000,16.275999,4.428016,4.555000,4.093416,16.756056,0.924436,0.272058,0.245465,...,0.811571,1.125259,1.125259,0.109713,1.573446,AGG_O_01,AGG,fr,none,0
AGG_O_01_P1,16.275999,0.001000,16.275000,5.646877,6.400000,3.805994,14.485587,0.674000,0.346905,-0.170667,...,0.838153,0.972110,0.972110,0.070207,0.761257,AGG_O_01,AGG,v0,none,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ULY_Y_96_P5,1.277456,6.454628,1.294826,1.519432,2.150591,1.458456,3.106510,0.608595,0.484396,1.262774,...,0.240840,0.448943,0.448943,0.698109,0.622618,ULY_Y_96,ULY,rnw,cv,1
ULY_Y_96_P5,1.273690,8.157261,1.284342,1.545650,2.531208,1.467207,3.153993,0.567536,0.486007,1.016162,...,0.247022,0.454578,0.454578,0.714566,0.606891,ULY_Y_96,ULY,iw,cv,1
ULY_Y_96_P5,1.273690,8.157261,1.284342,1.545650,2.531208,1.467207,3.153993,0.567536,0.486007,1.016162,...,0.276168,0.468649,0.468649,0.728377,0.601008,ULY_Y_96,ULY,fr,cv,1


In [78]:
variant = 'default'
grid_size = 5
grid_agg = 'cv'

default_df = df[(df['variant'] == variant) & (df['grid_size'] == 0)]
default_df = default_df.drop(columns=['min', 'variant', 'grid_size', 'grid_agg', 'site', 'site_type'])

gridded_df = df[(df['variant'] == variant) & (df['grid_size'] == grid_size) & (df['grid_agg'] == grid_agg)]
gridded_df = gridded_df.drop(columns=['min', 'variant', 'grid_size', 'grid_agg', 'site', 'site_type'])
gridded_df = gridded_df.add_prefix(f'xy_{grid_size}m_{grid_agg}|')

# Merge the two dataframes
merged_df = pd.concat([default_df, gridded_df], axis=1)

merged_corr = merged_df.corr(method="spearman", numeric_only=True)
# Get correlations with max height and sort them
max_height_corr = merged_corr["max"].abs().sort_values(ascending=False)

# Reorder the correlation matrix based on this ordering
ordered_corr = merged_corr.loc[max_height_corr.index, max_height_corr.index]


# Reset index to make it work with px.imshow
ordered_corr_reset = ordered_corr.reset_index()

fig = px.imshow(
    ordered_corr.values,
    x=ordered_corr.columns,
    y=ordered_corr.index,
    color_continuous_scale="RdBu_r",
    color_continuous_midpoint=0,
    title="Correlation Matrix",
    aspect="auto",
)

fig.update_traces(
    hovertemplate="<b>%{x}</b> vs <b>%{y}</b><br>Correlation: %{z:.3f}<extra></extra>"
)

fig.update_layout(
    width=1400,
    height=1200,
    xaxis=dict(tickfont=dict(size=8)),
    yaxis=dict(tickfont=dict(size=8)),
)
fig.show()